### Przedmiot: Python 2
### Temat: "Programowanie sieciowe"
### Imię nazwisko: Eduard Povierin
### Grupa: 2TD12A

---

## Cel:
Celem ćwiczenia jest zapoznanie z funkcjami sieciowymi standardowych bibliotek Pythona oraz
bibliotek requests oraz websockets

### Zadanie 1
Utworzyć serwer i klienta przy wykorzystaniu biblioteki socket i protokołu UDP.

In [ ]:
# Server
import socket

host = "localhost"
port = 1234

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.bind((host, port))

while True:
    data, address = server.recvfrom(1024)
    server.sendto(data, address)

In [ ]:
# Client
import socket

host = "localhost"
port = 1234

client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.sendto("Hey, from client!".encode(), (host, port))

data, address = client.recvfrom(1024)
print(data.decode())

### Zadanie 2
Utworzyć serwer i klienta przy wykorzystaniu biblioteki socket i protokołu TCP.

In [ ]:
# Server
import socket

host = "localhost"
port = 1234

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((host, port))
server.listen(1)

while True:
    connection, address = server.accept()
    data = connection.recv(1024)
    connection.sendall(data)
    connection.close()

In [ ]:
# Client
import socket

host = "localhost"
port = 1234

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((host, port))

client.sendall("Hehehe...".encode())
data = client.recv(1024)
print(data.decode())

client.close()

### Róznice między UDP i TCP

- Ustanowienie połączenia: Kod używa metody ```listen()``` do nasłuchiwania przychodzących połączeń i metody ```accept()``` do akceptowania przychodzącego połączenia. Oznacza to, że serwer oczekuje, aż klient ustanowi połączenie, zanim zacznie przyjmować i przetwarzać dane. W poprzednim kodzie z użyciem UDP serwer nie oczekuje na ustanowienie połączenia i może przyjmować dane niezależnie od obecności klienta.

- Wiarygodność dostawy: Kod używa metody ```sendall()``` do wysyłania danych, która gwarantuje, że wszystkie dane zostaną wysłane, oraz metody ```recv()``` do odbierania danych. W poprzednim kodzie z użyciem UDP dane są wysyłane za pomocą metody ```sendto()```, a serwer odbiera dane za pomocą metody ```recvfrom()```. W protokole UDP nie ma gwarancji, że wszystkie dane zostaną dostarczone, a kolejność ich dostarczania nie jest zagwarantowana.

- Zamykanie połączenia: Kod używa metody ```close()``` do zamykania połączenia po przetworzeniu danych. W poprzednim kodzie z użyciem UDP połączenie nie jest zamykane explicite, ponieważ protokół UDP nie wymaga ustanawiania połączenia.

### Zadanie 3
Utworzyć serwer i klienta przy wykorzystaniu biblioteki websocket, przetestować działanie z testem online

In [ ]:
# Server
import websockets
import asyncio

async def main():
    server = await websockets.serve(handler, "localhost", 8080)
    await server.wait_closed()

async def handler(websocket, path):
    await websocket.send("Let's STUDY!!!!")

    data = await websocket.recv()
    print(data)


asyncio.run(main())

In [ ]:
# Client 
import websockets
import asyncio

async def main():
    async with websockets.connect("ws://localhost:8080") as websocket:
        await websocket.send("Nice idea!")

        data = await websocket.recv()
        print(data)


asyncio.run(main())

### Zadanie 4
Pokazać na wybranych stronach działanie modułów biblioteki urllib.

In [2]:
import urllib.request

response = urllib.request.urlopen("http://google.com/")
code = response.getcode()
content = response.read().decode("utf-8")

print(code, content)

200 <!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="pl"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="ov8xV1ABHhbS_gPzPEoOuw">(function(){var _g={kEI:'cvNhZfyxA8Ooxc8PxcGiyA8',kEXPI:'0,1365468,206,4804,2316,383,1129371,1921,1576570,16114,28684,22430,1362,12319,4746,12834,4998,17075,38444,887,1985,2891,3926,213,7615,606,12205,48485,6397,9708,230,20573,4,57411,2216,4437,22595,6642,7596,1,42154,2,39761,5679,1020,31123,4567,6256,23421,1252,59701,8155,23351,873,19633,7,1922,5600,10043,36595,3142,17056,5797,10,243,14183,20206,27337,5403,3030,15816,1804,11491,2315,7205,5397,8585,276,11813,1632,2173,32443,16443,5208500,6942,2,580,132,50,5994203,2803214,3311,141,795,30154,160,2,2,2,2,2,16,23940228,4044106,14298,2374,43887,3,1599,3,503,3,1882,1392060,338513,388821,23031936,11584,1215,8409,2878,336,1259,4426,5799,

Widać że status jest 200, czyli wszystko jest w porządku. Widać też że jest to strona z kodem HTML.

### Zadanie 5 oraz 6
Utworzyć serwer HTTP z wykorzystaniem klasy BaseHTTPRequestHandler i HTTPServer.

In [ ]:
import os
import http.server

class MyRequestHandler(http.server.BaseHTTPRequestHandler):

    def do_GET(self):
        path = self.path

        if path == "/":
            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.end_headers()

            with open(os.path.join(os.path.dirname(__file__), "index.html"), "rb") as f:
                self.wfile.write(f.read())
        else:
            try:
                with open(path, "rb") as f:
                    self.send_response(200)
                    self.send_header("Content-type", f.info.get("Content-Type"))
                    self.end_headers()
                    self.wfile.write(f.read())
            except FileNotFoundError:
                self.send_response(404)
                self.send_header("Content-type", "text/html")
                self.end_headers()
                self.wfile.write(b"<h1>Not found :0</h1>")


server = http.server.HTTPServer(("localhost", 8000), MyRequestHandler)
print("Server started on http://localhost:8000")
server.serve_forever()

In [ ]:
# Przykładowy HTML
<!DOCTYPE html>
<html lang="en">
   <head>
      <meta charset="UTF-8" />
      <meta name="viewport" content="width=device-width, initial-scale=1.0" />
      <title>My Landing Page</title>
   </head>
   <style>
      body {
         background-color: #111;
         font-family: Arial, Helvetica, sans-serif;
         font-size: 1.2rem;
         color: #f4f4f4;
      }
      h1 {
         text-align: center;
         font-size: 3rem;
         margin-top: 2rem;
      }
      p {
         text-align: center;
         font-size: 1.5rem;
         margin-top: 2rem;
      }
   </style>
   <body>
      <h1>Welcome to my landing page!</h1>
      <p>Thanks for visiting. Please check back soon for updates.</p>
   </body>
</html>


### Zadanie 7
Zapoznać się z biblioteką requests i pokazać na różnych przykładach jej możliwości.

Robi polączenie do API ze zdięciami piesów i zapisuje je do pliku.

In [ ]:
import requests

response = requests.get("https://random.dog/woof.json")

if response.status_code == 200:
    data = response.json()
    image_url = data["url"]

    image_response = requests.get(image_url)
    if image_response.status_code == 200:
        with open("random_dog.jpg", "wb") as file:
            file.write(image_response.content)
        print("Image downloaded successfully!")
    else:
        print("Failed to download the image.")
else:
    print("Failed to fetch data from the API.")

### Zadanie 8
Pokazać dostęp do serwera FTP z wykorzystaniem biblioteki ftplib.

In [5]:
from ftplib import FTP

ftp_host = "demo.wftpserver.com"
ftp_user = "demo"
ftp_password = "demo"

ftp = FTP()

ftp.connect(ftp_host)

ftp.login(user=ftp_user, passwd=ftp_password)

print(ftp.getwelcome())

file_list = ftp.nlst()
print(f"List of files: {file_list}")

ftp.quit()

220 Wing FTP Server ready...
List of files: ['upload', 'download']


'221 Goodbye.'

## Wniosek:
Zapoznałem się z podstawami pracy w sieci w Pythonie, nauczyłem się tworzyć proste serwery i klientów oraz korzystać z bibliotek do pracy z siecią. Chciałbym również zaznaczyć, że wcześniej wykonywałem podobne zadania w języku JavaScript, dlatego łatwiej mi było zrozumieć, jak pracować z siecią w Pythonie.

## Dziekuje za uwage!